### Importacion de Biblioteca

Aquí importaremos todas las bibliotecas necesarias para el análisis de sentimientos y la clasificación de comentarios.

In [56]:
import praw
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

### Descarga de recursos de NLTK

 Descargaremos los recursos necesarios de NLTK, como stopwords y el lexicon para análisis de sentimientos.

In [57]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juand\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\juand\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\juand\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

### Inicialización de Reddit y Obtención de comentarios

Aquí inicializaremos la API de Reddit utilizando la biblioteca PRAW. Ademas, recopilaremos comentarios de Reddit de un subreddit sobre la salud publica en USA específico y los almacenaremos en una lista.


In [58]:
# Inicializar Reddit
reddit = praw.Reddit(
    client_id='vkbyNxlrdz1Wy9DNxfxnzA',
    client_secret='sC4RfUNF95ThEF0BtuTyfD1sV6DyWA',
    user_agent='my-app by u/Witty-Cause-3665'
)

# Subreddit: health
subreddit_2 = reddit.subreddit('health')
PublHealth_posts = subreddit_2.search("Community Health Centers USA", limit=20)

# Preprocesamiento de texto
stop_words = set(stopwords.words('english'))  # Palabras comunes que no aportan mucho significado
lemmatizer = WordNetLemmatizer()
sia = SentimentIntensityAnalyzer()

# Recolectar comentarios y etiquetas de sentimiento
comments_data_2 = []
sentiments = []

### Preprocesamiento de texto y Análisis de sentimientos
Limpiaremos y procesaremos los comentarios antes de realizar el análisis de sentimientos. Por ultimo, Usaremos el SentimentIntensityAnalyzer de NLTK para realizar el análisis de sentimientos.

In [59]:
def preprocess_text(text):
   
    text = text.lower()
   
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    tokens = word_tokenize(text)
    
    tokens = [word for word in tokens if word not in stop_words]
    
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

def analyze_sentiment(text):
    sentiment_score = sia.polarity_scores(text)
    return sentiment_score

for post in PublHealth_posts:
    post.comments.replace_more(limit=None)
    for comment in post.comments.list():
        comments_data_2.append(comment.body)
        preprocessed_comment = preprocess_text(comment.body)
        sentiment = analyze_sentiment(preprocessed_comment)
        sentiments.append(sentiment['compound'])



### Análisis de sentimientos


Utilizaremos el SentimentIntensityAnalyzer de NLTK para llevar a cabo el análisis de sentimientos. Posteriormente, aplicaremos el preprocesamiento y el análisis de sentimientos a los comentarios recopilados. Después, procederemos a crear un DataFrame de pandas para almacenar los comentarios procesados y sus etiquetas de sentimiento. Seguidamente, dividiremos los datos en conjuntos de entrenamiento y prueba. Luego, emplearemos TF-IDF para convertir los comentarios en características numéricas. Acto seguido, entrenaremos un modelo SVM para la clasificación de sentimientos. Una vez completado el entrenamiento, realizaremos predicciones en el conjunto de prueba y evaluaremos el rendimiento del modelo. Finalmente, crearemos un DataFrame para mostrar los comentarios, tanto el sentimiento real como el sentimiento predicho.

In [60]:

# Convertir etiquetas continuas en clases discretas
sentiment_labels_2 = ['Positive' if score > 0.05 else 'Negative' if score < -0.05 else 'Neutral' for score in sentiments]

# Crear DataFrame con los comentarios y sus etiquetas de sentimiento
data_2 = pd.DataFrame({'Comment Body': comments_data_2, 'Sentiment': sentiment_labels_2})

X_train, X_test, y_train, y_test = train_test_split(data_2['Comment Body'], data_2['Sentiment'], test_size=0.2, random_state=42)


tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


svm_model = SVC(kernel='linear')
svm_model.fit(X_train_tfidf, y_train)


y_pred = svm_model.predict(X_test_tfidf)

PublicHealth_comments_df = pd.DataFrame({'Comment Body': X_test, 'Actual Sentiment': y_test, 'Predicted Sentiment': y_pred})

PublicHealth_comments_df

,Comment Body,Actual Sentiment,Predicted Sentiment
162,So they finally did something about the mold?,Neutral,Negative
42,"/u/middleagedouchebag has a good reference, a...",Positive,Positive
90,"I'm all support for this community, but I'm ju...",Positive,Positive
60,[removed],Neutral,Neutral
114,Nobody ends a sentence with a semicolon.,Positive,Positive
137,This was just posted today. [Michio Kaku talk...,Neutral,Positive
41,"If you are serious the movie ""Children Undergr...",Negative,Negative
15,Shocker ;),Negative,Positive
113,I think part of the aim here is to de-stigmati...,Negative,Negative
108,Don't bring your grammar to their place of fee...,Positive,Negative


In [61]:
PublicHealth_comments_df.to_csv('PublicHealth_comments_df.csv', index=False)